### Price Optimization Machine Learning Model

##### Objective
We want to leverage a machine learning model to help us set optimal prices, the aim would be to increase revenue and/or margin while keeping in mind market conditions and customer trust.

##### Why Now?
I believe we are now well-positioned to design a price optimization model. We have complete access to all our current and historical Brightpearl data, including the pricing information we need.

##### Expected Benefits
- Revenue Uplift: with optimized pricing we can expect improved revenue performance per product
- Margin Protection: An optimized model could help us avoid underpricing
- Insights: Clear understanding of demand elasticity by product & segment

##### Scope
TBC

##### Data Needed

- Historical prices & sales (SKU × date/time × channel)
- Product costs
- Inventory & stockouts
- Promotions & discounts
- Competitor prices ????? Is this achieveable for us
- External demand drivers (seasonality, events)

##### Resources

- Tools: Data warehouse (Perceptium), Python ML stack, Tableau BI dashboard.

### Breakdown of Data from Tables

##### Historical prices & sales (SKU × date/time × channel) / Product costs
Order Table:
- ord_id - Order ID
- ord_invoicetaxDate - Tax Date
- ord_channelId - Channel ID
- ord_orderTypeCode - Type code (used to filter, example: PC or SC is a refund????? Please confirm)

Orderline Table:
- orl_ord_id - Order ID (Number for overall order)
- orl_id - OrderLine ID (Number for orderline, used to show individual lines inside of an order)
- orl_productSku - product SKU
- orl_productId - Product ID
- orl_nominalCode - For filtering (Not needed as a column)
- orl_itemCostValue - Cost (cost price for single unit of product)
- orl_quantity - Quantity (number of items purchased)
- orl_productPriceValue - Price (price of the product at the time the order is placed)
- DO NOT USE - orl_discountPercentage - discount percent on row (not dependable) 

In [2]:
# Imports Libraries - Remove unneeded 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
import pyodbc
warnings.filterwarnings('ignore')


from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose  

from sklearn.model_selection import train_test_split

##### Loading Dataset

In [3]:
#Load datasets - Original Dataset use sql below for full model
#orders = pd.read_csv('Order.csv')

# --- Step 1: Read the credentials from the text file ---
credentials = {}
try:
    with open('credentials.txt', 'r') as file:
        for line in file:
            # Remove leading/trailing whitespace and split the line at the first '='
            key, value = line.strip().split('=', 1)
            credentials[key.strip()] = value.strip()
except FileNotFoundError:
    print("Error: The 'credentials.txt' file was not found.")
    exit() # or handle the error in another way

# Assign credentials to variables
server_name = credentials.get('server')
database_name = credentials.get('database')
username = credentials.get('username')
password = credentials.get('password')
driver = '{ODBC Driver 17 for SQL Server}'

# Check for missing credentials
if not all([server_name, database_name, username, password]):
    raise ValueError("One or more credentials are missing from the file.")

# --- Step 2: Establish the connection ---
try:
    conn_string = (
        f'DRIVER={driver};'
        f'SERVER={server_name};'
        f'DATABASE={database_name};'
        f'UID={username};'
        f'PWD={password};'
    )
    conn = pyodbc.connect(conn_string)
    print("Connection to Azure SQL Database successful!")

except pyodbc.Error as ex:
    print(f"Error connecting to the database: {ex.args[0]}")
    conn = None

# --- Step 3: Fetch merged data and load into a single DataFrame ---
if conn:
    try:
        # SQL query to join the two tables - Use top(10000) for initial start until model is ready for larger dataset (this effects speed)
        # Note - WHERE o.ord_invoicetaxDate >= '2025-04-01' which shortens the reporting date for speed but on model complete increase data range
        merged_query = """
        SELECT DISTINCT --TOP(10000)
    o.ord_id AS [Order ID],
    o.ord_invoicetaxDate AS [Tax Date],
    o.ord_net AS [Net],
    o.ord_total AS [Total],
    o.ord_channelId AS [Channel Id],
    ord_orderTypeCode AS [Type Code],
    ol.orl_id AS [Orderline ID],
    ol.orl_productId AS [Product Id],
    ol.orl_productSku AS [Product SKU],
    ol.orl_productName AS [Product Name],
    ol.orl_quantity AS [Quantity],    
    CASE 
        WHEN ol.orl_compositionBundleParent = 1 THEN op.bpar_orl_calcRowNetValue
        WHEN ol.orl_compositionBundleChild = 1 THEN oc.bchd_orl_calcRowNetValue
        ELSE ol.orl_rowNetValue
    END AS [Product Value],
    CASE 
        WHEN ol.orl_compositionBundleParent = 1 THEN op.bpar_orl_calcRowTaxValue
        WHEN ol.orl_compositionBundleChild = 1 THEN oc.bchd_orl_calcRowTaxValue
        ELSE ol.orl_rowTaxValue
    END AS [Product Tax Value],
    ol.orl_productPriceValue AS [Price of Product],
    CASE 
        WHEN ol.orl_compositionBundleParent = 1 THEN op.bpar_orl_itemCostValue
        WHEN ol.orl_compositionBundleChild = 1 THEN oc.bchd_orl_itemCostValue
        ELSE ol.orl_itemCostValue
    END AS [Cost of Product],
    ol.orl_nominalCode AS [Nominal Code]
FROM dbo.tblOrder AS o
LEFT JOIN dbo.tblOrderLine AS ol ON o.ord_id = ol.orl_ord_id
LEFT JOIN Perceptium.tblOrderLineParentView AS op ON ol.orl_id = op.bpar_orl_id
LEFT JOIN Perceptium.tblOrderLineChildView AS oc ON ol.orl_id = oc.bchd_orl_id
WHERE o.ord_invoicetaxDate >= '2025-04-01' 
        """
        
        # Load the joined data directly into a single DataFrame
        orders = pd.read_sql(merged_query, conn)
        print(f"Successfully loaded {len(orders)} rows from the merged query.")
        #print("\nMerged DataFrame Head:")
        #print(orders.head())

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")

    finally:
        conn.close()
        print("Database connection closed.")
else:
    print("Cannot proceed with data fetching. Database connection failed.")

Connection to Azure SQL Database successful!
Successfully loaded 399470 rows from the merged query.
Database connection closed.


In [4]:
orders = orders.rename(columns={"Product SKU": "SKU"})
#Info
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399470 entries, 0 to 399469
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Order ID           399470 non-null  int64         
 1   Tax Date           399470 non-null  datetime64[ns]
 2   Net                399470 non-null  float64       
 3   Total              399470 non-null  float64       
 4   Channel Id         399470 non-null  int64         
 5   Type Code          399470 non-null  object        
 6   Orderline ID       399275 non-null  float64       
 7   Product Id         399275 non-null  float64       
 8   SKU                339431 non-null  object        
 9   Product Name       399275 non-null  object        
 10  Quantity           399275 non-null  float64       
 11  Product Value      398494 non-null  float64       
 12  Product Tax Value  398494 non-null  float64       
 13  Price of Product   399275 non-null  float64 

In [5]:
#Null values per columns

orders.isnull().sum()

Order ID                 0
Tax Date                 0
Net                      0
Total                    0
Channel Id               0
Type Code                0
Orderline ID           195
Product Id             195
SKU                  60039
Product Name           195
Quantity               195
Product Value          976
Product Tax Value      976
Price of Product       195
Cost of Product        976
Nominal Code           195
dtype: int64

### Promotion Data

We have a couple of CSV's with some promotional data that would be benefical to apply to this notebook

In [6]:
# Summer Sale 2025 - 25th of June to 27th of August
summerSale25 = pd.read_csv("C:/Users/Devin Ferko/Desktop/Codes/Machine Learning Projects/Price Optimization/Summer Sale 2025 - Prepped.csv")
summerSale25.head()

,Sale Type,SKU,Name,Summer - Sale Price DR,Summer - Sale Price TW,Summer - Sale Price OR,% off list DR,% off list TW,% off list OR
0,NaN,CO713DC,Crosswater Cucina Cook Industrial Style Single...,NaN,189.99,NaN,NaN,0.0500,NaN
1,NaN,CO713DF,Crosswater Cucina Cook Industrial Style Single...,NaN,234.99,NaN,NaN,0.0600,NaN
2,NaN,CO713DM,Crosswater Cucina Cook Industrial Style Single...,NaN,234.99,NaN,NaN,0.0600,NaN
3,NaN,CO721DC,Crosswater Cucina Cook Pull Out Single Lever K...,NaN,184.99,NaN,NaN,0.0263,NaN
4,NaN,CO721DM,Crosswater Cucina Cook Pull Out Single Lever K...,NaN,209.99,NaN,NaN,0.0455,NaN


In [7]:
# Spring Sale 2025 - March 5th to April 7th
springSale25 = pd.read_csv("C:/Users/Devin Ferko/Desktop/Codes/Machine Learning Projects/Price Optimization/Spring Sale 2025 - prepped.csv")
springSale25.head()

,Sale type,SKU,Name,Spring - Sale Price DR,Spring - Sale Price TW,Spring - Sale Price OR,% off list DR,% off list TW,% off list OR
0,Overstock,NOT-109FS/A-220-C/P,Vado Notion Wall Mounted Single Lever Basin Mi...,314.99,314.99,NaN,NaN,0.04,NaN
1,Overstock,R1SV-CHR,-,399.99,399.99,NaN,NaN,NaN,NaN
2,NaN,dr-1700p-reinforced-bath-pack-1,(DC) Drench P Shaped Reinforced Shower Bath & ...,479.99,479.99,NaN,NaN,0.02,NaN
3,NaN,dr-1700p-rh-reinforced-bath-pk-1,-,479.99,479.99,NaN,NaN,NaN,NaN
4,Overstock,TPM1CM/,Rangemaster Parma Kitchen Mixer Tap - Chrome,NaN,189.99,NaN,NaN,0.10,NaN


### Indicate if sale was present and apply discount percentages accordingly

In [8]:
# Add Sale Boolean Column and the discount percentage applied

# --- SUMMER SALE 2025 ---
# Define date windows
sum_start_date = pd.to_datetime('2025-06-25')
sum_end_date = pd.to_datetime('2025-08-27')

#Ensure clean dtypes - types for SKU are string and no trailing space
orders["SKU"] = orders["SKU"].astype(str).str.strip()
summerSale25["SKU"] = summerSale25["SKU"].astype(str).str.strip()

#Add summer sale 2025 column
orders["Summer_Sale"] = (
    orders["Tax Date"].between(sum_start_date, sum_end_date) #True if date falls in between
    & orders["SKU"].isin(summerSale25["SKU"]) #True if SKU matches
).astype("int8") # converts to boolean - t/f or 1/0

#Bring in discount columns from the sale sheet
#Keep only the columns we need from the sale table
discount_cols = ["% off list TW", "% off list DR", "% off list OR"]
orders = orders.merge(
    summerSale25[["SKU"] + discount_cols],
    on="SKU",
    how="left"
)


# Map Channel Id -> the corresponding discount column.
channel_discount_map = {
    "2": "% off list TW",
    "7": "% off list DR",
    "8": "% off list OR"
}

channel_key = orders["Channel Id"].astype(str).str.strip() # Ensure Channel Id's are strings
chosen_col = channel_key.map(channel_discount_map)  # per-row column name to use

# Vectorized pick of the right discount per row
disc_df = orders[discount_cols] #seperates discount values to seperate df
col_indexer = pd.Index(discount_cols).get_indexer(chosen_col) # converts column names in chosen_col into numeric indices so we can index the DataFrame efficiently.
row_indexer = np.arange(len(orders)) #array of row numbers [0, 1, 2, ..., n-1]

result = np.full(len(orders), np.nan, dtype=float) #empty array to hold discount values
in_window = orders["Tax Date"].between(sum_start_date, sum_end_date) #True if in window
valid_choice = col_indexer >= 0 #True if valid discount column exists for channel
mask = in_window & valid_choice #only select discount for orders in window and with valid channel

# Pull the values only where in window and with a valid channel/discount
result[mask] = disc_df.to_numpy()[row_indexer[mask], col_indexer[mask]]
orders["sumsale25_discount_percent"] = result

#If you prefer 0 instead of NaN when not applicable, uncomment:
orders["sumsale25_discount_percent"] = orders["sumsale25_discount_percent"].fillna(0)

#Drops unwanted columns
orders = orders.drop(['% off list TW', '% off list DR', '% off list OR'], axis=1)

# --- SPRING SALE 2025 ---
# Define date windows
spr_start_date = pd.to_datetime('2025-03-05')
spr_end_date = pd.to_datetime('2025-04-07')

#Ensure clean dtypes - types for SKU are string and no trailing space
orders["SKU"] = orders["SKU"].astype(str).str.strip()
springSale25["SKU"] = springSale25["SKU"].astype(str).str.strip()

#Add summer sale 2025 column
orders["Spring_Sale"] = (
    orders["Tax Date"].between(spr_start_date, spr_end_date) #True if date falls in between
    & orders["SKU"].isin(springSale25["SKU"]) #True if SKU matches
).astype("int8") # converts to boolean - t/f or 1/0

#Bring in discount columns from the sale sheet
#Keep only the columns we need from the sale table
discount_cols = ["% off list TW", "% off list DR", "% off list OR"]
orders = orders.merge(
    springSale25[["SKU"] + discount_cols],
    on="SKU",
    how="left"
)


# Map Channel Id -> the corresponding discount column.
channel_discount_map = {
    "2": "% off list TW",
    "7": "% off list DR",
    "8": "% off list OR"
}

channel_key = orders["Channel Id"].astype(str).str.strip() # Ensure Channel Id's are strings
chosen_col = channel_key.map(channel_discount_map)  # per-row column name to use

# Vectorized pick of the right discount per row
disc_df = orders[discount_cols] #seperates discount values to seperate df
col_indexer = pd.Index(discount_cols).get_indexer(chosen_col) # converts column names in chosen_col into numeric indices so we can index the DataFrame efficiently.
row_indexer = np.arange(len(orders)) #array of row numbers [0, 1, 2, ..., n-1]

result = np.full(len(orders), np.nan, dtype=float) #empty array to hold discount values
in_window = orders["Tax Date"].between(spr_start_date, spr_end_date) #True if in window
valid_choice = col_indexer >= 0 #True if valid discount column exists for channel
mask = in_window & valid_choice #only select discount for orders in window and with valid channel

# Pull the values only where in window and with a valid channel/discount
result[mask] = disc_df.to_numpy()[row_indexer[mask], col_indexer[mask]]
orders["sprsale25_discount_percent"] = result

#If you prefer 0 instead of NaN when not applicable, uncomment:
orders["sprsale25_discount_percent"] = orders["sprsale25_discount_percent"].fillna(0)

#Drops unwanted columns
orders = orders.drop(['% off list TW', '% off list DR', '% off list OR'], axis=1)

# --- TAX MONTH AND SEASONS ---
# Extract the month number from Tax Date
orders["TaxMonth"] = orders["Tax Date"].dt.month

# Flag if TaxMonth is in summer (June=6, July=7, August=8 for example)
orders["Winter"] = orders["TaxMonth"].isin([12, 1, 2]).astype(int)
orders["Spring"] = orders["TaxMonth"].isin([3, 4, 5]).astype(int)
orders["Summer"] = orders["TaxMonth"].isin([6, 7, 8]).astype(int)
orders["Fall"] = orders["TaxMonth"].isin([9, 10, 11]).astype(int)


In [9]:
# Uncomment the below if needed
#orders.info() 
#orders.to_csv('out.csv') 

### Product Attributes - Akeneo data

In [10]:
# Reads akeneo product attribute dataset
prdAttr = pd.read_csv("C:/Users/Devin Ferko/Desktop/Codes/Machine Learning Projects/Price Optimization/Akeneo Product Attributes - Sheet1.csv")
prdAttr.head()

,uuid,SKU,Family,Type DR,Type TW,Type OR,Basin Dimensions,Basin Tap Dimensions,Bath Dimensions,Bath Screen Dimensions,...,Toilet Dimensions,Vanity Unit Dimensions,Wall To Face Mm,Width Mm,Brand DR,Brand TW,Brand OR,LaunchDate DR,LaunchDate TW,LaunchDate OR
0,7ae7725d-1955-4c85-8646-b628d9298104,1000S8WETSET,Wetroom Panels,"Glass Panels,Walk In Shower Enclosures",Walk In Shower Enclosures,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1000.0,Harbour,Harbour,NaN,11/08/2019,11/08/2019,NaN
1,7da7c943-5775-44c3-a805-eb8fd5dbf637,100140001,Kitchen Sinks,NaN,Stainless Steel Kitchen Sinks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,465.0,NaN,Clearwater,NaN,NaN,NaN,NaN
2,f47929e3-492a-40ca-ae17-8afcf1206827,10045fp,Kitchen Sink Accessories,NaN,Splashbacks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1000.0,NaN,iGlass,NaN,NaN,NaN,NaN
3,8b7e314a-c57a-45d2-8be3-af9cc2180300,10045ms,Kitchen Sink Accessories,NaN,Splashbacks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1000.0,NaN,iGlass,NaN,NaN,NaN,NaN
4,c72ef444-c5b8-4968-84b2-6906a7f82927,10045ns,Kitchen Sink Accessories,NaN,Splashbacks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1000.0,NaN,iGlass,NaN,NaN,NaN,NaN
